# Combine All ATP Synthase and Proton Pump Models
---

In [2]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

from subsbml import createNewSubsystem, createSubsystem

Loading BokehJS ...

In [3]:
palette = colorcet.b_glasbey_category10
palette.remove( '#d62628')

## Combine ATP synthase tx, tl, membrane integration, atp synthesis models
---

In [33]:
# atp synthase models
ss1_atpsyn_txtl = createSubsystem('atpsynthase_tx_tl.xml', subsystemName = 'atpsyn_txtl')
ss2_atpsyn_mi = createSubsystem('atpsynthase_membrane_integration.xml', subsystemName = 'atpsyn_mi')
ss3_atpsynthesis = createSubsystem('atp_synthesis_through_atpsynthase.xml', subsystemName = 'atpsynthesis')

# proton pump ss
# ss4_pp_txtl = createSubsystem('proton_pump_tx_tl.xml', subsystemName = 'pp_txtl')
# ss5_pp_mi = createSubsystem('proton_pump_membrane_integration.xml', subsystemName = 'pp_mi')
# ss6_protontransport = createSubsystem('proton_transport_through_proton_pump.xml', subsystemName = 'protontransport')


In [67]:
#atp_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis
atp_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi 

In [68]:
atp_subsystem.getSpeciesByName('protein_atpsyn_b')

<Species protein_atpsyn_b_atpsyn_mibiocrnpyler_203770 "protein_atpsyn_b">

In [72]:
# spec_list_10 = ['metabolite_atp', 'proton_outside_H', 'dna_atpsyn', 'protein_RNAP_machinery',
#                'protein_Ribo_machinery', 'protein_RNAase_machinery']
spec_list_10 = ['dna_atpsyn', 'protein_RNAP_machinery',
               'protein_Ribo_machinery', 'protein_RNAase_machinery']
for i,spec in enumerate(spec_list_10):
    atp_subsystem.setSpeciesAmount(spec,10)


In [106]:
timepoints = np.linspace(0,10000,100)
results, _ = atp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

odeint failed with mxstep=500...

In [107]:
#ATP_id = atp_subsystem.getSpeciesByName('metabolite_atp').getId()
#ADP_id = atp_subsystem.getSpeciesByName('metabolite_adp').getId()

#outside_H _id = atp_subsystem.getSpeciesByName('proton_outside_H').getId()
#inside_H_id = atp_subsystem.getSpeciesByName('proton_inside_H').getId()
atp_synthase_b_protein_id = atp_subsystem.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = atp_subsystem.getSpeciesByName('protein_atpsyn').getId()

rna_atpsyn_id = atp_subsystem.getSpeciesByName('rna_atpsyn').getId()

dna_atpsyn_id = atp_subsystem.getSpeciesByName('dna_atpsyn').getId()

In [108]:
p = bokeh.plotting.figure(width = 450, height = 300)
# plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_b_protein_id]
# plot_str = ['atp', 'outside_H', 'inside_H', 'atp synthase bound']
plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id, rna_atpsyn_id]
plot_str = ['bound atp synthase', 'free atp synthase', 'rna atpsyn']
for i,item in enumerate(plot_list):
    p.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
bokeh.io.show(p)


## Combine ATP synthase tx, tl, membrane integration, atp synthesis models & ATP USE 
---

In [39]:
ss4_atp_use = createSubsystem('atp_use.xml', subsystemName = 'atp_use')

In [40]:
atp_subsystem_w_use = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use

In [46]:
# Initial conditions
spec_list_10 = ['dna_atpsyn', 'protein_RNAP_machinery',
               'protein_Ribo_machinery', 'protein_RNAase_machinery']
for i,spec in enumerate(spec_list_10):
    atp_subsystem_w_use.setSpeciesAmount(spec,10)

# timepoints and get results
timepoints = np.linspace(0,200,100)
results, _ = atp_subsystem_w_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_atp').getId()
ADP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_adp').getId()

outside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_outside_H').getId()
inside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_inside_H').getId()
atp_synthase_b_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn').getId()


# PLOT
p_w_atp = bokeh.plotting.figure(width = 450, height = 300)
# plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_protein_id]
# plot_str = ['atp', 'outside_H', 'inside_H', 'atp synthase']
plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_w_atp.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
bokeh.io.show(p_w_atp)



Nice we see that there's just a lower steady state when we add atp use, as expected

But how come it's staying regenerated for ever. Once there's no more outside H, no flow in of protons so no ATP synthesis via atp synthase. But atp should still be used by the atp_use subsbml so why isn't atp degrading?

We need to get it to say, once there is 1 atp synthase, something happens.

For the 'SBML for ATP synthesis via ATP synthase membrane protein' we need to include ATP synthase!!